In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from the Excel workbook - Bank_marketing.xlsx

InputData = "Staffing+Data.xlsx"

Data = pd.read_excel(InputData,sheet_name='DemandData')
StaffAv = pd.read_excel(InputData,sheet_name='StaffAvailability')
Cost = pd.read_excel(InputData,sheet_name='Cost')
transcost = pd.read_excel(InputData,sheet_name='ServiceRate')

In [4]:
Data.head()

State Month  Demand
0     A   Jan    5240
1     A   Feb    4878
2     A   Mar    5942
3     A   Apr    2297
4     A   May    1992

In [5]:
StaffAv.head()

State Month    LB    UB  StaffAvPer
0     A   Jan  0.70  0.90        0.81
1     A   Feb  0.65  0.85        0.76
2     A   Mar  0.70  0.80        0.75
3     A   Apr  0.75  0.85        0.80
4     A   May  0.70  0.85        0.78

In [6]:
Cost.head()

State Month  AnnualSalary  MonthlySalary  UnitOutSourceCost
0     A   Jan         60000         5000.0                180
1     A   Feb         60000         5000.0                180
2     A   Mar         60000         5000.0                180
3     A   Apr         60000         5000.0                180
4     A   May         60000         5000.0                180

In [7]:
transcost.head()

MgAppServedPerMonth
0                   40

## Data pre-processing 

In [8]:
# Create the required Python data structures for indexes and parameters
Demand= Data.set_index(['State','Month'])['Demand'].to_dict()
Demand

{('A', 'Jan'): 5240,
 ('A', 'Feb'): 4878,
 ('A', 'Mar'): 5942,
 ('A', 'Apr'): 2297,
 ('A', 'May'): 1992,
 ('A', 'Jun'): 2275,
 ('A', 'Jul'): 5334,
 ('A', 'Aug'): 3371,
 ('A', 'Sep'): 3759,
 ('A', 'Oct'): 3529,
 ('A', 'Nov'): 4284,
 ('A', 'Dec'): 5183,
 ('B', 'Jan'): 4927,
 ('B', 'Feb'): 2628,
 ('B', 'Mar'): 2974,
 ('B', 'Apr'): 2338,
 ('B', 'May'): 4020,
 ('B', 'Jun'): 3147,
 ('B', 'Jul'): 4271,
 ('B', 'Aug'): 2620,
 ('B', 'Sep'): 4517,
 ('B', 'Oct'): 4155,
 ('B', 'Nov'): 3137,
 ('B', 'Dec'): 4227,
 ('C', 'Jan'): 1162,
 ('C', 'Feb'): 1967,
 ('C', 'Mar'): 1898,
 ('C', 'Apr'): 2261,
 ('C', 'May'): 2030,
 ('C', 'Jun'): 1642,
 ('C', 'Jul'): 2489,
 ('C', 'Aug'): 2496,
 ('C', 'Sep'): 922,
 ('C', 'Oct'): 2421,
 ('C', 'Nov'): 963,
 ('C', 'Dec'): 1998}

In [9]:
FTE_Salary=Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()
FTE_Salary

{('A', 'Jan'): 5000.0,
 ('A', 'Feb'): 5000.0,
 ('A', 'Mar'): 5000.0,
 ('A', 'Apr'): 5000.0,
 ('A', 'May'): 5000.0,
 ('A', 'Jun'): 5000.0,
 ('A', 'Jul'): 5000.0,
 ('A', 'Aug'): 5000.0,
 ('A', 'Sep'): 5000.0,
 ('A', 'Oct'): 5000.0,
 ('A', 'Nov'): 5000.0,
 ('A', 'Dec'): 5000.0,
 ('B', 'Jan'): 4583.333333333333,
 ('B', 'Feb'): 4583.333333333333,
 ('B', 'Mar'): 4583.333333333333,
 ('B', 'Apr'): 4583.333333333333,
 ('B', 'May'): 4583.333333333333,
 ('B', 'Jun'): 4583.333333333333,
 ('B', 'Jul'): 4583.333333333333,
 ('B', 'Aug'): 4583.333333333333,
 ('B', 'Sep'): 4583.333333333333,
 ('B', 'Oct'): 4583.333333333333,
 ('B', 'Nov'): 4583.333333333333,
 ('B', 'Dec'): 4583.333333333333,
 ('C', 'Jan'): 4416.666666666667,
 ('C', 'Feb'): 4416.666666666667,
 ('C', 'Mar'): 4416.666666666667,
 ('C', 'Apr'): 4416.666666666667,
 ('C', 'May'): 4416.666666666667,
 ('C', 'Jun'): 4416.666666666667,
 ('C', 'Jul'): 4416.666666666667,
 ('C', 'Aug'): 4416.666666666667,
 ('C', 'Sep'): 4416.666666666667,
 ('C', 'Oc

In [10]:
Unitoutsourcecost=Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()
Unitoutsourcecost

{('A', 'Jan'): 180,
 ('A', 'Feb'): 180,
 ('A', 'Mar'): 180,
 ('A', 'Apr'): 180,
 ('A', 'May'): 180,
 ('A', 'Jun'): 180,
 ('A', 'Jul'): 180,
 ('A', 'Aug'): 180,
 ('A', 'Sep'): 180,
 ('A', 'Oct'): 180,
 ('A', 'Nov'): 180,
 ('A', 'Dec'): 180,
 ('B', 'Jan'): 150,
 ('B', 'Feb'): 150,
 ('B', 'Mar'): 150,
 ('B', 'Apr'): 150,
 ('B', 'May'): 150,
 ('B', 'Jun'): 150,
 ('B', 'Jul'): 150,
 ('B', 'Aug'): 150,
 ('B', 'Sep'): 150,
 ('B', 'Oct'): 150,
 ('B', 'Nov'): 150,
 ('B', 'Dec'): 150,
 ('C', 'Jan'): 160,
 ('C', 'Feb'): 160,
 ('C', 'Mar'): 160,
 ('C', 'Apr'): 160,
 ('C', 'May'): 160,
 ('C', 'Jun'): 160,
 ('C', 'Jul'): 160,
 ('C', 'Aug'): 160,
 ('C', 'Sep'): 160,
 ('C', 'Oct'): 160,
 ('C', 'Nov'): 160,
 ('C', 'Dec'): 160}

In [11]:
FTE_AppServRate=transcost.iloc[0,0]
FTE_AppServRate

40

In [12]:
StaffAva_LB=StaffAv.set_index(['State','Month'])['LB'].to_dict()
StaffAva_UB=StaffAv.set_index(['State','Month'])['UB'].to_dict()
StaffAvPer=StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()

In [13]:
StaffAvPer

{('A', 'Jan'): 0.81,
 ('A', 'Feb'): 0.76,
 ('A', 'Mar'): 0.75,
 ('A', 'Apr'): 0.8,
 ('A', 'May'): 0.78,
 ('A', 'Jun'): 0.73,
 ('A', 'Jul'): 0.68,
 ('A', 'Aug'): 0.76,
 ('A', 'Sep'): 0.81,
 ('A', 'Oct'): 0.73,
 ('A', 'Nov'): 0.68,
 ('A', 'Dec'): 0.65,
 ('B', 'Jan'): 0.81,
 ('B', 'Feb'): 0.76,
 ('B', 'Mar'): 0.75,
 ('B', 'Apr'): 0.8,
 ('B', 'May'): 0.78,
 ('B', 'Jun'): 0.73,
 ('B', 'Jul'): 0.68,
 ('B', 'Aug'): 0.76,
 ('B', 'Sep'): 0.81,
 ('B', 'Oct'): 0.73,
 ('B', 'Nov'): 0.68,
 ('B', 'Dec'): 0.65,
 ('C', 'Jan'): 0.81,
 ('C', 'Feb'): 0.76,
 ('C', 'Mar'): 0.75,
 ('C', 'Apr'): 0.8,
 ('C', 'May'): 0.78,
 ('C', 'Jun'): 0.73,
 ('C', 'Jul'): 0.68,
 ('C', 'Aug'): 0.76,
 ('C', 'Sep'): 0.81,
 ('C', 'Oct'): 0.73,
 ('C', 'Nov'): 0.68,
 ('C', 'Dec'): 0.65}

In [14]:
#Index
BankLoc = Cost['State'].unique()
Month = Cost['Month'].unique()

In [15]:
BankLoc

array(['A', 'B', 'C'], dtype=object)

In [16]:
Month

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

In [17]:
BankLoc.tolist()

['A', 'B', 'C']

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [18]:
# Creating a model instance
model=ConcreteModel()


In [19]:
# Define Pyomo sets and Parameters
model.i = Set(initialize=BankLoc.tolist(), doc="States")
model.j = Set(initialize=Month.tolist(), doc="Months")

In [20]:
model.i

In [21]:
#Defining parameters for demand,FTE Salaries
model.d = Param(model.i, model.j, initialize=Demand, doc = 'Demand')
model.s = Param(model.i, model.j, initialize=FTE_Salary, doc = 'FTE_Salary')
model.oc = Param(model.i, model.j, initialize=Unitoutsourcecost, doc = 'Unitoutsourcecost')
model.sa = Param(model.i, model.j, initialize=StaffAvPer, doc = 'StaffAv')

In [22]:
model.r = Param(initialize=FTE_AppServRate, doc ="FTE App Serve Rate")

In [23]:
# Constraints
def c_FTEAPP(model, i, j):
    return model.r * model.sa[i,j]
model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc ="NO OF FTE APP")

In [24]:
def Reg_A(model, i, j):
    return 0.30* model.d[i,j]
model.Reg_A = Param(model.i, model.j, initialize=Reg_A, doc ="RegResr_A")

In [25]:
def Reg_B(model, i, j):
    return 0.40* model.d[i,j]
model.Reg_B = Param(model.i, model.j, initialize=Reg_B, doc ="RegResr_B")

In [26]:
#Decision vraiables
model.x = Var(model.i,model.j, domain=NonNegativeReals, doc="No of FTE")
model.y = Var(model.i,model.j, domain=NonNegativeIntegers, doc="No of Outsourced App")

In [27]:
#Demand Constraint
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr = model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])
        

In [28]:
#Regulatory Constraint
model.regulatoryCons=ConstraintList()
for i in model.i:
    for j in model.j:
        if i =='A':
            model.regulatoryCons.add(expr=model.y['A',j] <= model.Reg_A['A',j])
        elif i =='B':
            model.regulatoryCons.add(expr=model.y['B',j] <= model.Reg_B['B',j])

In [29]:
# Objective function
def objective_rule(model):
    return sum(model.s[i,j] * model.x[i,j] for i in model.i for j in model.j) +\
           sum(model.oc[i,j] * model.y[i,j] for i in model.i for j in model.j)

model.objective=Objective(rule=objective_rule, sense=minimize, doc="Define Objective Function")

In [30]:
# Invoking the solver
result_cost= SolverFactory('glpk').solve(model)
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05584216117858887
# ----------------------------------------------------------
#   Solution Information
# ------------

In [31]:
#dispalying optimal result parameters
model.pprint()

13 Set Declarations
    FTEAPPNO_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    Reg_A_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), (

In [32]:
# Print the value of the objective function
#dispalying optimal result parameters
model.objective()

17962336.448769882

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [33]:
#Displaying results
#value of variable x
#no of fte for state A Dec month
print(model.x['A','Dec'].value)
#noof applications processed by outsourced vendor
print(model.y['A','Dec'].value)


139.576923076923
1554.0


In [34]:
#To print total cost i.e final solution
Total_cost=model.objective.expr()
Total_cost

17962336.448769882

In [35]:
# Creating dataframe for the results

FTE_Staff=[]
for i in BankLoc:
    for j in Month:
        dem=Demand[i,j]
        v=model.x[i,j].value#no of FTE
        w=model.y[i,j].value#no of outsource APP
        no_of_FTE_processed_app = model.x[i,j].value*40*model.sa[i,j]
        PercentFTEprocessed = round(model.x[i,j].value*40*model.sa[i,j] / Demand[i,j]*100,2)
        PercentOutsourced=round(model.y[i,j].value / Demand[i,j]*100,2)
        ProcessingCost=round((model.x[i,j].value*FTE_Salary[i,j]/FTE_AppServRate/StaffAvPer[i,j])\
                        +(model.y[i,j].value*Unitoutsourcecost[i,j]),2)
        FTE_Staff.append([i,j,dem,v,w,no_of_FTE_processed_app,PercentFTEprocessed,PercentOutsourced,ProcessingCost])
FTE_Staff= pd.DataFrame(FTE_Staff, columns=['State','Month','dem','FTE','App_outsourced','no_of_FTE_processed_app','PercentFTEprocessed','PercentOutsourced','ProcessingCost'])     
FTE_Staff

State Month   dem         FTE  App_outsourced  no_of_FTE_processed_app  \
0      A   Jan  5240  161.728395             0.0                   5240.0   
1      A   Feb  4878  160.460526             0.0                   4878.0   
2      A   Mar  5942  198.066667             0.0                   5942.0   
3      A   Apr  2297   71.781250             0.0                   2297.0   
4      A   May  1992   63.846154             0.0                   1992.0   
5      A   Jun  2275   77.910959             0.0                   2275.0   
6      A   Jul  5334  137.279412          1600.0                   3734.0   
7      A   Aug  3371  110.888158             0.0                   3371.0   
8      A   Sep  3759  116.018519             0.0                   3759.0   
9      A   Oct  3529  120.856164             0.0                   3529.0   
10     A   Nov  4284  110.257353          1285.0                   2999.0   
11     A   Dec  5183  139.576923          1554.0                   3629.0   
12     B   Jan  4927  152.067901             0.0                   4927.0   
13     B   Feb  2628   51.875000          1051.0                   1577.0   
14     B   Mar  2974   59.500000          1189.0                   1785.0   
15     B   Apr  2338   73.062500             0.0                   2338.0   
16     B   May  4020  128.846154             0.0                   4020.0   
17     B   Jun  3147   64.691781          1258.0                   1889.0   
18     B   Jul  4271   94.227941          1708.0                   2563.0   
19     B   Aug  2620   51.710526          1048.0                   1572.0   
20     B   Sep  4517  139.413580             0.0                   4517.0   
21     B   Oct  4155   85.376712          1662.0                   2493.0   
22     B   Nov  3137   69.227941          1254.0                   1883.0   
23     B   Dec  4227   97.576923          1690.0                   2537.0   
24     C   Jan  1162   35.864198             0.0                   1162.0   
25     C   Feb  1967   64.703947             0.0                   1967.0   
26     C   Mar  1898   63.266667             0.0                   1898.0   
27     C   Apr  2261   70.656250             0.0                   2261.0   
28     C   May  2030   65.064103             0.0                   2030.0   
29     C   Jun  1642   56.232877             0.0                   1642.0   
30     C   Jul  2489    0.000000          2489.0                      0.0   
31     C   Aug  2496   82.105263             0.0                   2496.0   
32     C   Sep   922   28.456790             0.0                    922.0   
33     C   Oct  2421   82.910959             0.0                   2421.0   
34     C   Nov   963    0.000000           963.0                      0.0   
35     C   Dec  1998    0.000000          1998.0                      0.0   

    PercentFTEprocessed  PercentOutsourced  ProcessingCost  
0                100.00               0.00        24958.09  
1                100.00               0.00        26391.53  
2                100.00               0.00        33011.11  
3                100.00               0.00        11215.82  
4                100.00               0.00        10231.76  
5                100.00               0.00        13340.92  
6                 70.00              30.00       313235.19  
7                100.00               0.00        18238.18  
8                100.00               0.00        17904.09  
9                100.00               0.00        20694.55  
10                70.00              30.00       251567.90  
11                70.02              29.98       306561.72  
12               100.00               0.00        21511.66  
13                60.01              39.99       165471.07  
14                60.02              39.98       187440.28  
15               100.00               0.00        10464.68  
16               100.00               0.00        18927.72  
17                60.03              39.97       198854

In [ ]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("C:\\Users\\hp\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("C:\\Users\\hp\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data'
FTE_Staff.to_excel(writer, sheet_name='Question 2')

# save the excel file
writer.save()
print('DataFrame is loaded successfully to Excel Sheet.')

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [ ]:
# Creating a model instance
model=ConcreteModel()


In [ ]:
# Define Pyomo sets and Parameters
model.i = Set(initialize=BankLoc.tolist(), doc="States")
model.j = Set(initialize=Month.tolist(), doc="Months")

In [ ]:
# Decision variables
#Defining parameters for demand,FTE Salaries
model.d = Param(model.i, model.j, initialize=Demand, doc = 'Demand')
model.s = Param(model.i, model.j, initialize=FTE_Salary, doc = 'FTE_Salary')
model.oc = Param(model.i, model.j, initialize=Unitoutsourcecost, doc = 'Unitoutsourcecost')
model.sa = Param(model.i, model.j, initialize=StaffAvPer, doc = 'StaffAv')

#worst case scenario
model.salb = Param(model.i, model.j, initialize=StaffAva_LB, doc = 'StaffAva_LB')

In [ ]:
model.r = Param(initialize=FTE_AppServRate, doc ="FTE App Serve Rate")

In [ ]:
# Constraints
def c_FTEAPP(model, i, j):
    return model.r * model.salb[i,j]
model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc ="NO OF FTE APP")

In [ ]:
def Reg_A(model, i, j):
    return 0.30* model.d[i,j]
model.Reg_A = Param(model.i, model.j, initialize=Reg_A, doc ="RegResr_A")

In [ ]:
def Reg_B(model, i, j):
    return 0.40* model.d[i,j]
model.Reg_B = Param(model.i, model.j, initialize=Reg_B, doc ="RegResr_B")

In [ ]:
#Decision vraiables
model.x = Var(model.i,model.j, domain=NonNegativeReals, doc="No of FTE")
model.y = Var(model.i,model.j, domain=NonNegativeIntegers, doc="No of Outsourced App")

In [ ]:
#Demand Constraint
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr = model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])
        

In [ ]:
#Regulatory Constraint
model.regulatoryCons=ConstraintList()
for i in model.i:
    for j in model.j:
        if i =='A':
            model.regulatoryCons.add(expr=model.y['A',j] <= model.Reg_A['A',j])
        elif i =='B':
            model.regulatoryCons.add(expr=model.y['B',j] <= model.Reg_B['B',j])

In [ ]:
# Objective function

def objective_rule(model):
    return sum(model.s[i,j] * model.x[i,j] for i in model.i for j in model.j) +\
           sum(model.oc[i,j] * model.y[i,j] for i in model.i for j in model.j)

model.objective1=Objective(rule=objective_rule, sense=minimize, doc="Define Objective Function")

In [ ]:
# Invoking the solver
result_cost= SolverFactory('glpk').solve(model)
result_cost.write()

In [ ]:
# Print the value of the objective function
model.objective1()

In [ ]:
#To print total cost i.e final solution
Total_cost_1=model.objective1.expr()
Total_cost_1

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [ ]:
# Creating dataframe for the results
FTE_Staff_worst=[]
for i in BankLoc:
    for j in Month:
        dem=Demand[i,j]
        v=model.x[i,j].value
        w=model.y[i,j].value
        no_of_FTE_processed_app = model.x[i,j].value*40*model.sa[i,j]
        PercentFTEprocessed = round(model.x[i,j].value*40*model.sa[i,j] / Demand[i,j]*100,2)
        PercentOutsourced=round(model.y[i,j].value / Demand[i,j]*100,2)
        ProcessingCost=round((model.x[i,j].value*FTE_Salary[i,j]/FTE_AppServRate/StaffAvPer[i,j])\
                        +(model.y[i,j].value*Unitoutsourcecost[i,j]),2)
        avg_cost_per_app = round( ProcessingCost/dem,2)
        FTE_Staff_worst.append([i,j,dem,v,w,PercentOutsourced,ProcessingCost,avg_cost_per_app])
FTE_Staff_worst= pd.DataFrame(FTE_Staff_worst, columns=['State','Month','dem','FTE','App_outsourced','PercentOutsourced','ProcessingCost','avg_cost_per_app'])      
FTE_Staff_worst

#### 3.1.2 Percentage of outsourced applications 

In [ ]:
# write your code here
#calculate avg FTE per month
TotalFTE=FTE_Staff_worst['FTE'].sum()
AvgFTEpermonth= (TotalFTE/12).round(2)
AvgFTEpermonth

In [ ]:
#Total no of outsourced apps
#Total application demand
TotalDemand = Data['Demand'].sum()
TotalDemand

In [ ]:
# Per centage of outsourced application
PercentageOutsourced=((FTE_Staff_worst['App_outsourced'].sum()/TotalDemand)*100).round(2)
PercentageOutsourced

#### 3.1.3 Average cost per application

In [ ]:
# write your code here
AVG=(Total_cost_1 / TotalDemand)
AVG

In [ ]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("C:\\Users\\hp\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("C:\\Users\\hp\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data'
FTE_Staff_worst.to_excel(writer, sheet_name='worst_case Scenario')

# save the excel file
writer.save()
print('DataFrame is loaded successfully to Excel Sheet.')

### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [ ]:
# Creating a model instance
model=ConcreteModel()

In [ ]:
# Define Pyomo sets and Parameters
model.i = Set(initialize=BankLoc.tolist(), doc="States")
model.j = Set(initialize=Month.tolist(), doc="Months")

In [ ]:
# Decision variables
#Defining parameters for demand,FTE Salaries
model.d = Param(model.i, model.j, initialize=Demand, doc = 'Demand')
model.s = Param(model.i, model.j, initialize=FTE_Salary, doc = 'FTE_Salary')
model.oc = Param(model.i, model.j, initialize=Unitoutsourcecost, doc = 'Unitoutsourcecost')
model.sa = Param(model.i, model.j, initialize=StaffAvPer, doc = 'StaffAv')

#best case scenario
model.saub = Param(model.i, model.j, initialize=StaffAva_UB, doc = 'StaffAva_UB')

In [ ]:
model.r = Param(initialize=FTE_AppServRate, doc ="FTE App Serve Rate")

In [ ]:
# Constraints
def c_FTEAPP(model, i, j):
    return model.r * model.saub[i,j]
model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc ="NO OF FTE APP")

In [ ]:
def Reg_A(model, i, j):
    return 0.30* model.d[i,j]
model.Reg_A = Param(model.i, model.j, initialize=Reg_A, doc ="RegResr_A")

In [ ]:
def Reg_B(model, i, j):
    return 0.40* model.d[i,j]
model.Reg_B = Param(model.i, model.j, initialize=Reg_B, doc ="RegResr_B")

In [ ]:
#Decision vraiables
model.x = Var(model.i,model.j, domain=NonNegativeReals, doc="No of FTE")
model.y = Var(model.i,model.j, domain=NonNegativeIntegers, doc="No of Outsourced App")

In [ ]:
#Demand Constraint
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr = model.x[i,j]*model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])

In [ ]:
#Regulatory Constraint
model.regulatoryCons=ConstraintList()
for i in model.i:
    for j in model.j:
        if i =='A':
            model.regulatoryCons.add(expr=model.y['A',j] <= model.Reg_A['A',j])
        elif i =='B':
            model.regulatoryCons.add(expr=model.y['B',j] <= model.Reg_B['B',j])

In [ ]:
# Objective function

def objective_rule(model):
    return sum(model.s[i,j] * model.x[i,j] for i in model.i for j in model.j) +\
           sum(model.oc[i,j] * model.y[i,j] for i in model.i for j in model.j)

model.objective2=Objective(rule=objective_rule, sense=minimize, doc="Define Objective Function")

In [ ]:
# Invoking the solver
result_cost= SolverFactory('glpk').solve(model)
result_cost.write()

In [ ]:
# Print the value of the objective function
model.objective2()

In [ ]:
#To print total cost i.e final solution
Total_cost_2=model.objective2.expr()
Total_cost

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [ ]:
# Creating dataframe for the results
FTE_Staff_best=[]
for i in BankLoc:
    for j in Month:
        dem=Demand[i,j]
        v=model.x[i,j].value
        w=model.y[i,j].value
        no_of_FTE_processed_app = model.x[i,j].value*40*model.sa[i,j]
        PercentFTEprocessed = round(model.x[i,j].value*40*model.sa[i,j] / Demand[i,j]*100,2)
        PercentOutsourced=round(model.y[i,j].value / Demand[i,j]*100,2)
        ProcessingCost=round((model.x[i,j].value*FTE_Salary[i,j]/FTE_AppServRate/StaffAvPer[i,j])\
                        +(model.y[i,j].value*Unitoutsourcecost[i,j]),2)
        FTE_Staff_best.append([i,j,dem,v,w,no_of_FTE_processed_app,PercentFTEprocessed,PercentOutsourced,ProcessingCost])
FTE_Staff_best= pd.DataFrame(FTE_Staff_best, columns=['State','Month','dem','FTE','App_outsourced','no_of_FTE_processed_app','PercentFTEprocessed','PercentOutsourced','ProcessingCost'])  
FTE_Staff_best

In [ ]:
# Writing the results in to an Excel sheet

# Importing the required library
from openpyxl import load_workbook

book = load_workbook("C:\\Users\\hp\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx")

# create excel writer object
writer = pd.ExcelWriter("C:\\Users\\hp\\Desktop\\Optimisation Case Study - Staff Planning\\Staffing+Data.xlsx", engine = 'openpyxl')

#Assigning the workbook to the writer object
writer.book = book

# write dataframe to excel sheet named 'Staffing+Data'
FTE_Staff_best.to_excel(writer, sheet_name='best_case Scenario')

# save the excel file
writer.save()
print('DataFrame is loaded successfully to Excel Sheet.')

#### 3.2.2 Percentage of outsourced applications

In [ ]:
#calculate avg FTE per month
TotalFTE=FTE_Staff_best['FTE'].sum()
AvgFTEpermonth= (TotalFTE/12).round(2)
AvgFTEpermonth

In [ ]:
#Total no of outsourced apps
#Total application demand
TotalDemand = Data['Demand'].sum()
TotalDemand

In [ ]:
# Per centage of outsourced application
PercentageOutsourced=((FTE_Staff_best['App_outsourced'].sum()/TotalDemand)*100).round(2)
PercentageOutsourced

#### 3.2.3 Average cost per application

In [ ]:
# write your code here
AVG=(Total_cost_2 / TotalDemand)
AVG

# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

In [ ]:
import jovian

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df = FTE_Staff[['Month', 'Demand', 'w',"no_of_FTE_processed_app"]].groupby(['Month']).sum()
df[["Percentage of outsource app"]] = df['w']/df['Demand']*100
df[['Percentage of FTE processed app']] = df['no_of_FTE_processed_app']/df['Demand']*100
ax = df[['Percentage of outsource app','Percentage of FTE processed app']].plot(kind='bar', stacked=True, figsize=(10,10))
ax_set_ylabel("Percentage", fontsize=12)
plt.title("Stacked bar chart between percentages of FTE Processed app and Outsourced app", fontsize=12)
plt.show()